# Tutorial \#1: Hello World

The **Universal library** is a ready-to-use header-only library that provides plug-in replacement for native types, and provides a low-friction environment to start exploring alternatives to IEEE floating-point in your own algorithms. 

In order to use it in this online environment you need to first import the global header file `universal.hpp` which is not part of the library itself

In [1]:
#include "universal.hpp"

Let's implement a simple kernel that computes the product of two numbers

In [2]:
template<typename Real>
Real MyKernel(const Real& a, const Real& b) {
    return a * b;  // replace this with your kernel computation
}

Next, we call this kernel for built-in `double` data type

In [3]:
constexpr double pi = 3.14159265358979323846;
double a = sqrt(2.2);
double b = pi;
std::cout << "Result: " << MyKernel(a, b) << std::endl;

Result: 4.65973


Let us now include the top-level header file of the `posit` data type

In [4]:
#include <universal/number/posit/posit.hpp>

and perform the same computation with `posit`s instead of `double`s

In [5]:
constexpr double pi = 3.14159265358979323846;
using Real = sw::universal::posit<32,2>;
Real a = sqrt(2.2);
Real b = pi;
std::cout << "Result: " << MyKernel(a, b) << std::endl;

Result: 4.65973


One complicated technical aspect of floating-point arithmetic is the problem of `catastrophic cancellation`, which is the problem when precision and dynamic range of the values that partake in the computation are not compatible. 

First let's pull in some BLAS functionality to make the math a little easier.

In [6]:
#include <universal/blas/blas.hpp>

Let's create a parameterized method that will allow us to test this cancellation dynamic with different number systems.

In [7]:
template<typename Scalar>
void catastrophicCancellationTest() {
	std::cout << "\nScalar type : " << typeid(Scalar).name() << '\n';
	using Matrix = sw::universal::blas::matrix<Scalar>;

	Scalar a1 = 3.2e8;
	Scalar a2 = 1;
	Scalar a3 = -1;
	Scalar a4 = 8e7;
	Matrix A = { 
		{ a1, a2, a3, a4 }, 
		{ a4, a3, a2, a1 },
	};
	Scalar b1 = 4.0e7;
	Scalar b2 = 1;
	Scalar b3 = -1;
	Scalar b4 = -1.6e8;
	Matrix B = {
		{ b1, b4 },
		{ b2, b3 },
		{ b3, b2 },
		{ b4, b1 }
	};

	std::cout << std::setprecision(10);
	std::cout << "matrix A: \n" << A << '\n';
	std::cout << "matrix B: \n" << B << '\n';
	auto C = A * B;
	std::cout << "matrix C: \n" << C << '\n';
	if (C[0][0] == 2 && C[1][1] == 2) {
		std::cout << "PASS\n";
	}
	else {
		std::cout << "FAIL\n";
	}
}

In [8]:
catastrophicCancellationTest<float>();  // FAILS due to catastrophic cancellation


Scalar type : f
matrix A: 
320000000.0000000000 1.0000000000 -1.0000000000 80000000.0000000000 
80000000.0000000000 -1.0000000000 1.0000000000 320000000.0000000000 

matrix B: 
40000000.0000000000 -160000000.0000000000 
1.0000000000 -1.0000000000 
-1.0000000000 1.0000000000 
-160000000.0000000000 40000000.0000000000 

matrix C: 
0.0000000000 -48000000449314816.0000000000 
-48000000449314816.0000000000 0.0000000000 

FAIL


In [9]:
catastrophicCancellationTest<double>(); // FAILS due to catastrophic cancellation


Scalar type : d
matrix A: 
320000000.0000000000 1.0000000000 -1.0000000000 80000000.0000000000 
80000000.0000000000 -1.0000000000 1.0000000000 320000000.0000000000 

matrix B: 
40000000.0000000000 -160000000.0000000000 
1.0000000000 -1.0000000000 
-1.0000000000 1.0000000000 
-160000000.0000000000 40000000.0000000000 

matrix C: 
0.0000000000 -48000000000000000.0000000000 
-48000000000000000.0000000000 0.0000000000 

FAIL


In [10]:
catastrophicCancellationTest< sw::universal::posit<32,2> >(); // PASSES due to FDP


Scalar type : N2sw9universal5positILm32ELm2EEE
matrix A: 
320000000 1 -1 80000000 
80000000 -1 1 320000000 

matrix B: 
40000000 -160000000 
1 -1 
-1 1 
-160000000 40000000 

matrix C: 
2 -4.800027962e+16 
-4.800027962e+16 2 

PASS


The IEEE-754 `float` and `double` computations demonstrate catastrophic cancellation because the dynamic range between the large product and the small product terms is larger than can be covered by the fraction bits, that is, the range is `> 2^53`.

The 32-bit posit computation does not exhibit catastrophic cancellation because in the matrix product, it uses a fused dot product which does NOT round during the sum of products, and thus correctly deals with the large difference in dynamic range between the terms of interest.